# Title

## Subtitle

Text zu:
- was ist MeinBGE
- Wer ist MeinBGE
- Warum Analyse?

## Datengrundlage

Text:
- was sind das für Daten? (nicht öffentlich, etc)
- Was mache ich mit den Daten?
- Welche mittel/module nutze ich?

## Explorative Analyse

### Allgemeine Übersicht 

There are to this date <i>(Zeitpunkt Postgres-Dump) Anzahl User*innen</i> and <i>Anzahl Kinder</i> with <i>Anzahl Crowdhörnchen</i> users as Crowdhörnchen (monatliche Lastschrift). <br>
These people have now generated about <i>Spendentopf-Summe (kumulativ)</i> which led to <i>Anzahl Grundeinkommen</i> distributed over <i>Anzahl Verlosungen</i> since <i>Startdatum</i>. <br>
In the last years <i>Anzahl Erwachsene mit GE</i> adult users and <i>Anzahl Kinder mit GE</i> children received a monthly GE for one year.

### Getting to know the users

#### Allgemein
- Woher kommen User*innen? --> Karte
- Wie alt sind User*innen?

#### Zeitverlauf
- Wann kommen User *innen dazu? Wann gehen User*innen?
    - Bargraph mit (Gewinn/Verlust und Absolutwert als Line)
- Wann werden User*innen zu CH? Wann hören sie auf?
    - Bargraph mit (Gewinn/Verlust und Absolutwert als Line)
- Da wenig persönliche Daten erhoben werden, kaum Charakterisierung der User*innen möglich

### Donations

- wie viel Spenden pro Monat? Gruppiert nach Spendentopf/Verein
- Welche Menschen spenden?
    - Einteilbar in Altersgruppen?
- tageswerte mit events:
    - gruppierung in Neu-CH, CH-Update, Einzelspende (area-chart, absolut), mit markierung von Events (v.A. Newsletter, wenn möglich auch andere Aktionen

### Raffles

- Wie viel nicht-CH nehmen mehr als einmal teil? (als KPI?)
- Wie viele GE pro Monat?

### Zusätzlich wenn Zeit

- Lebenszyklus User mit mittelwerten
    1. Account-Erstellung
        - mit Dauerauftrag?
    2. Verlosungsteilnahme
        - weitere Teilnahmen?
        - Account löschung?
    3. CH-Sign-Up
        - Wie lange? 
        - Wie viel Teilnahmen an Verlosungen vor Sign-Up?
    4. CH-Sign-Out
        - Zeitdauer bis erneutem CH-Sign-Up?
        - Wie häufig Sign-Up/Sign-Out?
    5. Account Löschung
        - Wie viele Menschen löschen Account?

- Wann werden Newsletter akzeptiert?

- Wünsche:
    - Was steht im Wunsch? --> Bag of Words (häufigste Wortkombination aus 2-3 Worten)
    - Welche Wünsche werden am meisten geliked? (Auch auf Website dargestellt)

- Kommentare auf Website-Magazin/Blogposts:
    - Sentiment-Analyse
